In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.7 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
import random
import tensorflow as tf

# Set random seeds for reproducibility
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# Load dataset
sheet_id = "1j_Euo80PrGckVDVr2hTG9zZebxJD0TSC"
sheet_name = "Sheet1"
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df = pd.read_csv(csv_url)

# Preprocessing
df = df.set_index('Date')
df = df.drop(columns=["YEAR", "MO", "DY"])
target_column = "WS10M"
scaler = MinMaxScaler()
df[target_column] = scaler.fit_transform(df[[target_column]])
series = df[target_column].values

# Function to create sequences
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

# Random search parameter space
param_space = {
    'filters': [32, 64, 96, 128],
    'kernel_size': [1, 2, 3, 4, 5],
    'lr': [1e-4, 3e-4, 1e-3, 3e-3, 1e-2],
    'epochs': [10, 15, 20, 25, 30],
    'batch_size': [8, 16, 32]
}

# Loop over lags (window sizes) from 1 to 14
results = []

for lag in range(1, 15):
    print(f"\n\n### LAG WINDOW: {lag} ###")

    # Create sequences
    X, y = create_sequences(series, lag)
    split_index = int(len(X) * 0.8)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    # Reset random seed for each lag to ensure consistent comparison
    np.random.seed(random_seed)
    random.seed(random_seed)
    tf.random.set_seed(random_seed)

    # Random search implementation
    best_val_loss = float('inf')
    best_params = None

    # Number of random trials
    n_trials = 20

    for trial_num in range(n_trials):
        # Set trial-specific seed for reproducible random sampling
        trial_seed = random_seed + lag * 100 + trial_num
        random.seed(trial_seed)

        # Sample random parameters
        params = {
            'filters': random.choice(param_space['filters']),
            'kernel_size': random.choice([k for k in param_space['kernel_size'] if k <= lag]),
            'lr': random.choice(param_space['lr']),
            'epochs': random.choice(param_space['epochs']),
            'batch_size': random.choice(param_space['batch_size'])
        }

        tscv = TimeSeriesSplit(n_splits=3)
        val_losses = []

        for train_idx, val_idx in tscv.split(X_train):
            X_t, X_v = X_train[train_idx], X_train[val_idx]
            y_t, y_v = y_train[train_idx], y_train[val_idx]

            # Reset model weights for each fold
            tf.keras.backend.clear_session()
            np.random.seed(trial_seed)
            tf.random.set_seed(trial_seed)

            # Create model with current parameters
            model = Sequential()
            model.add(Conv1D(filters=params['filters'],
                             kernel_size=params['kernel_size'],
                             activation='relu',
                             input_shape=X_train.shape[1:]))
            model.add(Flatten())
            model.add(Dense(1))
            model.compile(optimizer=Adam(learning_rate=params['lr']),
                          loss='mse')

            model.fit(X_t, y_t,
                      epochs=params['epochs'],
                      batch_size=params['batch_size'],
                      verbose=0)

            val_loss = model.evaluate(X_v, y_v, verbose=0)
            val_losses.append(val_loss)

        mean_val_loss = np.mean(val_losses)

        if mean_val_loss < best_val_loss:
            best_val_loss = mean_val_loss
            best_params = params

    print(f"Best params for lag {lag}: {best_params}")

    # Train final model with best parameters
    tf.keras.backend.clear_session()
    np.random.seed(random_seed + lag * 1000)  # Different seed for final model
    tf.random.set_seed(random_seed + lag * 1000)

    model = Sequential()
    model.add(Conv1D(filters=best_params['filters'],
                     kernel_size=best_params['kernel_size'],
                     activation='relu',
                     input_shape=X_train.shape[1:]))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=best_params['lr']),
                  loss='mse')
    model.fit(X_train, y_train, epochs=best_params['epochs'],
              batch_size=best_params['batch_size'], verbose=0)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_inv = scaler.inverse_transform(y_pred)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Metrics
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100
    r2 = r2_score(y_test_inv, y_pred_inv)

    # Store results
    results.append({
        'Lag': lag,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R2': r2,
        'Params': best_params
    })

    # Plotting
    plt.figure(figsize=(12, 4))
    plt.plot(y_test_inv, label='Actual', linewidth=2)
    plt.plot(y_pred_inv, label='Predicted', linestyle='--')
    plt.title(f'CNN Forecast vs Actual (Lag {lag})')
    plt.xlabel("Time")
    plt.ylabel("WS10M")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Display all performance metrics
results_df = pd.DataFrame(results)
print("\nSummary of CNN model performance for each lag window:\n")
print(results_df)

Output hidden; open in https://colab.research.google.com to view.